$\newcommand{\mr}[1]{\mathrm{#1}}$ 
$\newcommand{\bmr}[1]{\mathit{\mathbf{#1}}}$ 
$\newcommand{\tr}{^\mathrm{T}}$
$\newcommand{\lbr}{\left\lbrace}$
$\newcommand{\rbr}{\right\rbrace}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\jj}{\mathrm{j}}$

## <center> Pomoćne funkcije
Interaktivni grafici koriste Python modul mpld3, koji se može instalirati komandom
    
    
    conda install mpld3
    
Primeri rade i ukoliko modul mpld3 nije instaliran, ali grafici neće biti interaktivni.

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
try:
    import mpld3
    mpld3.enable_notebook()
    print("Modul mpld3 je instaliran.")
except:
    print("Modul mpld3 nije instaliran.")
    
def plotSpectrum(x, xlabel = "", ylabel="", title="", show=True):
    n = len(x)
    plt.plot(np.fft.fftshift(np.fft.fftfreq(n)),np.fft.fftshift(abs(np.fft.fft(x))));
    plt.xlabel(xlabel); plt.ylabel(ylabel); plt.title(title);
    if show:
        plt.show()
    
def stemSpectrum(x, xlabel = "", ylabel="", title="", show=True):
    n = len(x)
    F = np.fft.fftshift(np.fft.fftfreq(n))
    X = np.fft.fftshift(abs(np.fft.fft(x)))
    plt.stem(F, X)#, use_line_collection=True);
    plt.xlabel(xlabel); plt.ylabel(ylabel); plt.title(title);
    if show:
        plt.show()

def plotSignal(x, xlabel = "", ylabel="", title="", show=True):
    plt.plot(x);
    plt.xlabel(xlabel); plt.ylabel(ylabel); plt.title(title);
    if show:
        plt.show()

def stemSignal(x, xlabel = "", ylabel="", title="", show=True):
    plt.stem(x)# ,use_line_collection=True);
    plt.xlabel(xlabel); plt.ylabel(ylabel); plt.title(title);
    if show:
        plt.show()
    

Modul mpld3 je instaliran.


In [2]:
def readSamples(fileName):
    samples = []
    inFile = open(fileName, "r")
    for line in inFile:
        samples.append( complex(line) )
    return np.array(samples)

In [3]:
fs = 61.44e6
testsignal = readSamples("testsignal.txt")
N = len(testsignal)
freqs = (np.arange(N)/N - 0.5) * fs/1e6
plt.plot(freqs, np.fft.fftshift(abs(np.fft.fft(testsignal))))
plt.xlabel("f [MHz]")
plt.ylabel("|X|");

In [4]:
M = 16
oversampled_signal = np.zeros(len(testsignal)*M,dtype=complex) # argument dtype=complex je obavezan!
oversampled_signal[::M] = testsignal
freqs = (np.arange(N*M)/(N*M) - 0.5) * M*fs/1e6
plt.plot(freqs, np.fft.fftshift(abs(np.fft.fft(oversampled_signal))))
plt.xlabel("f [MHz]")
plt.ylabel("|X|");

In [5]:
from remezlp import *

fmax = 25.0e6
fs = 61.44e6

Fpass = fmax/fs / M
Fstop = ((fs/2 -fmax)*2 + fmax)/fs/M
AdB = 60.0
Amax = 20.0*np.log10(1.0+10.0**(-AdB/20.0))
Amin = 20.0*np.log10(1.0-10.0**(-AdB/20.0))

deltaPass = 10.0**(-AdB/20.0)
deltaStop = deltaPass

plt.plot(freqs, np.fft.fftshift(abs(np.fft.fft(oversampled_signal))))
plt.plot([-M*fs/2/1e6, -M*Fstop*fs/1e6, -M*Fpass*fs/1e6, M*Fpass*fs/1e6, M*Fstop*fs/1e6, M*fs/2/1e6],[0, 0, 1, 1, 0, 0], 'r')
plt.xlabel("f [MHz]")
plt.ylabel("|X|");

In [6]:
h = remezlp( Fpass, Fstop, deltaPass, deltaStop, nPoints=8192*8,Nmax=500)
# Test signal se periodicno produzava za len(h)/2 odbiraka, koliko iznosi kasnjenje filtra.
oversampled_signal_perext = np.concatenate((oversampled_signal, oversampled_signal[0:int(len(h/2))]))
xi = signal.lfilter(h, 1, oversampled_signal_perext)
xi = xi[len(h):]
plt.plot(freqs, np.fft.fftshift(abs(np.fft.fft(xi))))
plt.plot([-M*fs/2/1e6, -M*Fstop*fs/1e6, -M*Fpass*fs/1e6, M*Fpass*fs/1e6, M*Fstop*fs/1e6, M*fs/2/1e6],[0, 0, 1, 1, 0, 0], 'r')
plt.xlabel("f [MHz]")
plt.ylabel("|X|");

TypeError: remez() takes 3 positional arguments but 4 were given

In [ ]:
XdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xi))))
plt.plot(freqs, XdB-np.max(XdB))
plt.ylim(-80,0);

In [ ]:
XdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xi* signal.hann(len(xi),'periodic')))))
plt.plot(freqs, XdB-np.max(XdB))
plt.ylim(-80,0);

In [ ]:
f0 = M*fs*3/8
#f0 = M*fs*1/8
Omega0 = 2 * np.pi * f0/(M*fs)
n = np.arange(len(xi))
cplx_sine = np.exp(1j * n * Omega0)
#cplx_sine = np.round(np.exp(1j * n * Omega0), 3) # konacna tacnost CORDIC-a
xi_shifted = xi * cplx_sine
#xi_shifted = np.round(xi * cplx_sine, 3) # konacna tacnost duzine binarne reci
XdB_shifted = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xi_shifted))))
plt.plot(freqs, XdB_shifted-np.max(XdB_shifted))
plt.ylim(-80,0);

In [ ]:
xr = xi_shifted.real
#XrdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xr* signal.hann(len(xi),'periodic')))))
XrdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xr))))
plt.plot(freqs, XrdB-np.max(XrdB))
plt.ylim(-80,0);

In [ ]:
NyqZones = 4
nrz = np.ones(NyqZones)
xr_upsampled = np.zeros((len(xr)-1)*NyqZones+1)
xr_upsampled[::NyqZones] = xr
xr_nrz = np.convolve(nrz, xr_upsampled)
XrnrzdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xr_nrz))))
freqs_nrz = (np.arange(N*M*len(nrz))/(N*M*len(nrz)) - 0.5) * M*fs*len(nrz)/1e6
plt.plot(freqs_nrz, XrnrzdB-np.max(XrnrzdB))
plt.plot(freqs, XrdB-np.max(XrdB),'r')
plt.ylim(-80,0);

In [ ]:
print(xr_upsampled[:17])
print(len(xr))

In [ ]:
NyqZones = 4
rf = np.concatenate([np.ones(int(NyqZones/2)), -1.0*np.ones(int(NyqZones/2))])
xr_rf = np.convolve(rf, xr_upsampled)
XrrfdB = 20*np.log10(np.fft.fftshift(abs(np.fft.fft(xr_rf))))
freqs_rf = (np.arange(N*M*len(rf))/(N*M*len(rf)) - 0.5) * M*fs*len(rf)/1e6
plt.plot(freqs_rf, XrrfdB-np.max(XrrfdB))
plt.plot(freqs_nrz, XrnrzdB-np.max(XrnrzdB),'r--')
plt.ylim(-80,0);

In [ ]:
plt.plot(freqs_rf, XrrfdB-np.max(XrrfdB))
plt.plot(freqs+M*fs/1e6, XrdB-np.max(XrdB),'r')
plt.ylim(-80,0);